In [1]:
import torch
import datasets
import matplotlib.pyplot as plt
from tft.transforms import WPT2D, IWPT2D 
from tft.utils import CompandGN, InvCompandGN, Quantize
from torchvision.transforms.v2 import CenterCrop, PILToTensor, ToPILImage
from pytorch_wavelets import DWTForward, DWTInverse

In [2]:
img = datasets.load_dataset('danjacobellis/LSDIR',split='validation')[28]['image']

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

In [3]:
x = PILToTensor()(CenterCrop(288)(img)).to(torch.float).unsqueeze(0)/127.5 - 1.0
wt = DWTForward(J=1, mode='periodization', wave='bior4.4')
iwt = DWTInverse(mode='periodization', wave='bior4.4')
J = 5
wpt = WPT2D(wt=wt, J=J)
iwpt = IWPT2D(iwt=iwt, J=J)

In [4]:
c = CompandGN(
    num_features=3*(4**J),
    num_groups=3*(4**J),
    compand_eps=0.1,
    compand_pow=0.5,
    rescale=5
)
d = InvCompandGN(c)

In [5]:
for bits in range(2,12):
    q = Quantize(bits=bits)
    q.eval()
    with torch.no_grad():
        X = q(c(wpt(x)))
        x_hat = iwpt(d(X))
    PSNR = -10*torch.nn.functional.mse_loss(x,x_hat).log10().item() + 6.02
    print(bits, PSNR)

2 10.364762861728668
3 15.199035425186157
4 23.267954607009887
5 30.369758625030517
6 36.369516868591305
7 42.567050476074215
8 48.842146415710445
9 55.123980064392086
10 60.795152206420894
11 66.97675468444824
